In [1]:
# Script to download the google news:

from GoogleNews import GoogleNews, AutoTokenizer, AutoModel
from datetime import timedelta, date
googlenews = GoogleNews()
googlenews = GoogleNews(lang='en')


def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2018, 1, 1)
end_date = date(2020, 8, 1)
for single_date in daterange(start_date, end_date):
    date3 = single_date.strftime("%m/%d/%Y")
    print(date3)
    googlenews.setTimeRange(date3,date3)   #mm/dd/yyyy
    googlenews.search('Bitcoin')
    googlenews.getpage(1)
    news=googlenews.gettext()
    print(news)
    googlenews.clear()
    with open('out_news2.txt', 'a') as f:  #I put the "2" to not erase the file if we execute this
        print(date3,news, file=f)
    

01/01/2018
["Japan's GDP Grows Due to Bitcoin Wealth Effect | Finance ...", 'Q&A: How is the growth of bitcoin affecting the environment?', 'Bitcoin fever to burn out in ‘spectacular crash,’ David Stockman warned', "'Privacy Coin' Verge is Allegedly Leaking Users' IP Addresses | Altcoins", 'Buying—and Selling—Property with Bitcoin', 'Ripple was the best-performing cryptocurrency of 2017, beating bitcoin', 'The Illogical Value Proposition Of Bitcoin', "Bitcoin's gender divide could be a bad sign, experts say", 'Bitcoin Adoption by Businesses in 2017', "Japan's GDP Grows Due to Bitcoin Wealth Effect | Finance ...", 'Q&A: How is the growth of bitcoin affecting the environment?', 'Bitcoin fever to burn out in ‘spectacular crash,’ David Stockman warned', "'Privacy Coin' Verge is Allegedly Leaking Users' IP Addresses | Altcoins", 'Buying—and Selling—Property with Bitcoin', 'Ripple was the best-performing cryptocurrency of 2017, beating bitcoin', 'The Illogical Value Proposition Of Bitcoin', 

['CES 2018: Kodak soars on KodakCoin and Bitcoin mining plans', "JP Morgan's Jamie Dimon Regrets his ”Bitcoin is a Fraud ...", "Dimon Says He Regrets Calling Bitcoin a 'Fraud'", 'Jamie Dimon says he regrets calling bitcoin a fraud and believes in the technology behind it', 'South Korea clampdown on bitcoin gathers pace', 'Bitcoin ETF Proposals Withdrawn After SEC Pushback - CoinDesk', 'The Bitcoin Price Equivalence Shows What “Cheap” Cryptocurrencies Are Really Worth | Economics', 'Breakingviews - Chancellor: Bitcoin will never be real money', 'The tea company that planned to buy bitcoin mining machines is backpedaling on its plans', 'Bitcoin is Not a Currency, Bank of Israel Deputy Governor States', 'CES 2018: Kodak soars on KodakCoin and Bitcoin mining plans', "JP Morgan's Jamie Dimon Regrets his ”Bitcoin is a Fraud ...", "Dimon Says He Regrets Calling Bitcoin a 'Fraud'", 'Jamie Dimon says he regrets calling bitcoin a fraud and believes in the technology behind it', 'South Korea clam

KeyboardInterrupt: 

In [1]:
# Script to set the google news list in the right format:

import ast
import pandas as pd


dates = []
news = []
lines = []

# Read file lines
with open('out_news.txt', 'r') as f:
    lines = f.readlines()

for l in lines:
    # Spliting date and news
    date_string, news_string = l.split(' ', maxsplit=1)
    # Safely evaluate news list
    news_list = ast.literal_eval(news_string)
    # Extend date the amount of news in that date
    dates.extend([date_string] * len(news_list))
    # Append news
    news.extend(news_list)

# Create dataframe for the dates and news
df = pd.DataFrame({'Date': dates, 'New': news})

# Print the dataframe
print(df)

             Date                                                New
0      01/01/2018  Japan's GDP Grows Due to Bitcoin Wealth Effect...
1      01/01/2018  Q&A: How is the growth of bitcoin affecting th...
2      01/01/2018  Bitcoin fever to burn out in ‘spectacular cras...
3      01/01/2018  'Privacy Coin' Verge is Allegedly Leaking User...
4      01/01/2018           Buying—and Selling—Property with Bitcoin
...           ...                                                ...
18558  07/31/2020  Bitcoin: Analysts warn of correction, $11,700 ...
18559  07/31/2020  Market Wrap: Bitcoin Pushes to $11,450, DeFi V...
18560  07/31/2020  Chinese Police Take Down $6 Billion Plustoken ...
18561  07/31/2020  Bitcoin Trading Is Booming in Uncertain Russia...
18562  07/31/2020          ‘Get out of paper money and into bitcoin’

[18563 rows x 2 columns]


In [ ]:
# Writting the news to a csv file:

df.to_csv ('googlenews.csv', index = False, header=True)

In [3]:
# We define the model to get the sentiment

from datetime import timedelta, date
from GoogleNews import GoogleNews
from transformers import pipeline, AutoTokenizer, AutoModel


df['New'] = df['New'].astype('str') 
tokenizer2 = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
labels = ['buy','sell']

In [4]:
classifier = pipeline('zero-shot-classification',model="nlptown/bert-base-multilingual-uncased-sentiment",tokenizer=tokenizer2)

In [ ]:
# We start to calculate the sentiment and we drop it on a file

for i in range (1,len(df)):
          
    with open('out_news_sentiment2.txt', 'a') as f:     #I put the "2" to not erase the file if we execute this
        
        print(df.iloc[i,0],df.iloc[i,1],classifier(df.iloc[i,1], labels), file=f)

In [122]:
# Feature engineering over the text with the sentiment:


data = pd.read_csv('out_news_sentiment.txt',
                   sep=" {'sequence': | 'scores':",
                   header=None,
                   engine='python')

data['Date']=data[0]
data['New']=data[1]
data['Sentiment']=data[2]

data=data.drop([0,1,2], axis=1)

data['New']=data['New'].str.slice(1, -29)

data['Sentiment']=data['Sentiment'].str.slice(2,-2)
data[['Buy', 'Sell']] = data['Sentiment'].str.split(', ', n=1, expand=True)
data=data.drop(['Sentiment'], axis=1)
data['Date'] = pd.to_datetime(data['Date'],format="%m/%d/%Y")
data['Buy'] = pd.to_numeric(data['Buy'])
data['Sell'] = pd.to_numeric(data['Sell'])
data.head()




,Date,New,Buy,Sell
0,2018-01-01,Q&A: How is the growth of bitcoin affecting th...,0.626632,0.373368
1,2018-01-01,Bitcoin fever to burn out in ‘spectacular cras...,0.520645,0.479355
2,2018-01-01,'Privacy Coin' Verge is Allegedly Leaking User...,0.524635,0.475365
3,2018-01-01,Buying—and Selling—Property with Bitcoin,0.628039,0.371961
4,2018-01-01,Ripple was the best-performing cryptocurrency ...,0.566184,0.433816


In [123]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18562 entries, 0 to 18561
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    18562 non-null  datetime64[ns]
 1   New     18562 non-null  object        
 2   Buy     18562 non-null  float64       
 3   Sell    18562 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 580.2+ KB


In [124]:
data.shape

(18562, 4)

In [125]:
data_mean_buy = data['Buy'].groupby([data['Date'].dt.year, data['Date'].dt.month]).mean()
data_mean_sell = data['Sell'].groupby([data['Date'].dt.year, data['Date'].dt.month]).mean()

In [126]:
data_mean_buy 

Date  Date
2018  1       0.567840
      2       0.571816
      3       0.569291
      4       0.567131
      5       0.573572
      6       0.573041
      7       0.572795
      8       0.574623
      9       0.574812
      10      0.576961
      11      0.566538
      12      0.574100
2019  1       0.568907
      2       0.580169
      3       0.575164
      4       0.573195
      5       0.576646
      6       0.575009
      7       0.570079
      8       0.574261
      9       0.573758
      10      0.571170
      11      0.570507
      12      0.574129
2020  1       0.572034
      2       0.566059
      3       0.566690
      4       0.568151
      5       0.568148
      6       0.565075
      7       0.561992
Name: Buy, dtype: float64

In [127]:
data_mean_sell

Date  Date
2018  1       0.432160
      2       0.428184
      3       0.430709
      4       0.432869
      5       0.426428
      6       0.426959
      7       0.427205
      8       0.425377
      9       0.425188
      10      0.423039
      11      0.433462
      12      0.425900
2019  1       0.431093
      2       0.419831
      3       0.424836
      4       0.426805
      5       0.423354
      6       0.424991
      7       0.429921
      8       0.425739
      9       0.426242
      10      0.428830
      11      0.429493
      12      0.425871
2020  1       0.427966
      2       0.433941
      3       0.433310
      4       0.431849
      5       0.431852
      6       0.434925
      7       0.438008
Name: Sell, dtype: float64

THE ZERO-SHOT CLASSIFICATION MODEL WITH THE LABELS "BUY" OR "SELL" DOES NOT SEEM TO WORK. WE WILL TRY THE SAME PROCESS USING A 5 STARS CLASSIFICATION MODEL

In [134]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline, AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

classifier = pipeline(task="sentiment-analysis", model=model,tokenizer=tokenizer, return_all_scores=True)

In [135]:
# Script to set the google news list in the right format:

import ast
import pandas as pd


dates = []
news = []
lines = []

# Read file lines
with open('out_news.txt', 'r') as f:
    lines = f.readlines()

for l in lines:
    # Spliting date and news
    date_string, news_string = l.split(' ', maxsplit=1)
    # Safely evaluate news list
    news_list = ast.literal_eval(news_string)
    # Extend date the amount of news in that date
    dates.extend([date_string] * len(news_list))
    # Append news
    news.extend(news_list)

# Create dataframe for the dates and news
df = pd.DataFrame({'Date': dates, 'New': news})

# Print the dataframe
print(df)

             Date                                                New
0      01/01/2018  Japan's GDP Grows Due to Bitcoin Wealth Effect...
1      01/01/2018  Q&A: How is the growth of bitcoin affecting th...
2      01/01/2018  Bitcoin fever to burn out in ‘spectacular cras...
3      01/01/2018  'Privacy Coin' Verge is Allegedly Leaking User...
4      01/01/2018           Buying—and Selling—Property with Bitcoin
...           ...                                                ...
18558  07/31/2020  Bitcoin: Analysts warn of correction, $11,700 ...
18559  07/31/2020  Market Wrap: Bitcoin Pushes to $11,450, DeFi V...
18560  07/31/2020  Chinese Police Take Down $6 Billion Plustoken ...
18561  07/31/2020  Bitcoin Trading Is Booming in Uncertain Russia...
18562  07/31/2020          ‘Get out of paper money and into bitcoin’

[18563 rows x 2 columns]


In [ ]:
# We start to calculate the sentiment and we drop it on a file

for i in range (1,len(df)):
          
    with open('out_news_sentiment_googlenews.txt2', 'a') as f:     #I put the "2" to not erase the file if we execute this
        
        print(df.iloc[i,0],df.iloc[i,1],classifier(df.iloc[i,1]), file=f)